# 3. Retrieve physical values in real time and apply a function

In this sample, we will focus on the following methods:

- Retrieve real time data from another edge
- Apply a function to the retrieved data
- Upload the processed data in real time

## Scenario
Use the iOS application **intdash Motion**.
Receive the streamed data from your iPhone, process the sensor data with `intdash-py`, and upload it to the server. In this scenario, we will use a moving average as an example of a function to be applied.

## Preparation

Before starting this scenario, prepare the following.

- An edge that performs measurement
- intdash Motion Application
- Signal definition for general sensor data

### Data to be used
In this scenario, the following data needs to be prepared on the server side.

| Data item | Data name that appears in this scenario |
|:---|:---|
|Edge for data acquisition|sdk_edge1|
|Edge for data upload|sdk_edge2|
| Signal definition(\*) | `sp_ACCX`, `sp_ACCY`, `sp_ACCZ`|

(\*) Use the same signal definition used in **"1. Retrieve time series data and save as CSV"**.


### Import packages and create a client
For `url` given to ` intdash.Client`, specify the environment of the intdash server. For `edge_token`, specify the token issued for the edge you use.
(\* Login with `username` and `password` is also possible, but it is recommended to use edge token for continued use.)

In [43]:
import pandas as pd

import intdash
from intdash import timeutils

# Create client
client = intdash.Client(
    url = "https://example.intdash.jp",
    edge_token = "your_token",
)

### Register the signal definition
Use the same signal definition used in **"1. Retrieve time series data and save as CSV"**.
See the following file for how to convert general sensor type data to numerical values.

[create-signal-general-sensor.ipynb](../0_create-signals/create-signal-general-sensor.ipynb)

In this sample, only the conversion definition of "acceleration" is registered.

### Confirm that the signal definitions are registered
Confirm that the signal definitions for this scenario are registered.

In [2]:
signals = client.signals.list(label='sp')

In [49]:
for s in signals:
    print(s.label,  end=', ')

sp_ACCX, sp_ACCY, sp_ACCZ, 

## Retrieve the edge

In [3]:
sdk_edge1 = client.edges.list(name='sdk_edge1')[0]
sdk_edge2 = client.edges.list(name='sdk_edge2')[0]

In [4]:
sdk_edge1.name, sdk_edge2.name

('sdk_edge1', 'sdk_edge2')

## Create a Queue
The process from download to upload is performed using a Queue.

In [5]:
import queue
q = queue.Queue(maxsize=5)

## Preparation of data retrieval (Downstream)
Define a process to receive time series data from an edge through the server.

### Create a request
For ``src_edge_uuid``, specify the source edge. In this example, the edge ``sdk_edge1`` running intdash Motion is used.
Specify the ``label`` name of the signal definition in `data_id` of `core.DataFilter`.

In [6]:
d_specs = [
        intdash.DownstreamSpec(
            src_edge_uuid = sdk_edge1.uuid, #edge_uuid
            filters = [
                 intdash.DataFilter(data_type=intdash.DataType.float.value, data_id='sp_ACCX',channel=1),  # Acceleration X
                 intdash.DataFilter(data_type=intdash.DataType.float.value, data_id='sp_ACCY',channel=1),  # Acceleration Y
                 intdash.DataFilter(data_type=intdash.DataType.float.value, data_id='sp_ACCZ',channel=1),  # Acceleration Z
            ],
        ),
    ]

### Define a function to be applied to the received data
Define a process to receive time series data and add it to the Queue as is.

In [7]:
# Put the received time-series data to the Queue.
def callback(unit):
    try:
        q.put_nowait(unit)
    except queue.Full:
        pass

## Preparation for data upload (Upstream)
Define a function that processes the received data and sends the data back to the server as new time series data.

### Create a request
Specify the UUID of the uploading edge.

In [9]:
u_specs = [
        intdash.UpstreamSpec(
            src_edge_uuid = sdk_edge2.uuid,
        )
    ]

### Define the process
In this scenario, define the process that calculates the moving average and returns it to the server.

In [10]:
import numpy as np

# The function calculate moving average.
def calc_ave(score, array, ave_num):
    array.append(score)
    if len(array) > ave_num:
        array.popleft()
  
    return  np.sum(array)/ len(array)

### Define an uploader function

Define the following processes:

- Get data from the Queue
- Switch the process depending on the data type of time series data
- Put data into the process
- Send the newly created data back (return the intdash.Unit with "yield")

In [11]:
AVE_NUM = 5

import struct
from collections import deque

acc_x_dq = deque([])
acc_y_dq = deque([])
acc_z_dq = deque([])


# Calculate moving average of the received time-series data,  convert it to Unit and upload. 
def upload_func():
    while True:
        try:
            unit = q.get_nowait()
            
            # Skip basetime.
            if unit.data.data_type.value == intdash.DataType.basetime.value:
                yield unit
                continue
            
            # Skip other data.
            if unit.data.data_type.value != intdash.DataType.float.value:
                yield unit
                continue
                
            # Get intdash.intdash.data.GeneralSensor 
            sensor_data = unit.data
                
            if unit.data.data_id == 'sp_ACCX':
                acc_x = unit.data.value
                ave_acc_x = calc_ave(acc_x, acc_x_dq, AVE_NUM)
                
                if ave_acc_x is None:
                    continue
               
                yield intdash.Unit(
                      elapsed_time = unit.elapsed_time,
                      channel = 1,
                      data =  intdash.data.Float(data_id = 'sp_ACCX', value =ave_acc_x ),
                    )
                continue
                
                 
            if unit.data.data_id == 'sp_ACCY':
                acc_y = unit.data.value
                ave_acc_y = calc_ave(acc_y, acc_y_dq, AVE_NUM)
                
                if ave_acc_y is None:
                    continue
               
                yield intdash.Unit(
                      elapsed_time = unit.elapsed_time,
                      channel = 1,
                      data =  intdash.data.Float(data_id = 'sp_ACCY', value =ave_acc_y ),
                    )
                continue
                
            if unit.data.data_id == 'sp_ACCZ':
                acc_z = unit.data.value
                ave_acc_z = calc_ave(acc_z, acc_z_dq, AVE_NUM)
                
                if ave_acc_z is None:
                    continue
                
                yield intdash.Unit(
                      elapsed_time = unit.elapsed_time,
                      channel = 1,
                      data = intdash.data.Float(data_id = 'sp_ACCZ', value =ave_acc_z ),
                    )
                
                continue
                
        except queue.Empty:
            yield

## Start stream processing

In [12]:
wsconn = client.connect_websocket()

### Start upstream

In [13]:
wsconn.open_upstreams(
    specs = u_specs,
    iterators = [upload_func()],
)

### Start downstream

In [14]:
wsconn.open_downstreams(
    specs = d_specs,
    callbacks = [callback],
)

## Disconnect
If you want to end the process, be sure to execute the following to disconnect.

In [15]:
wsconn.close()

## Check data with Visual M2M Data Visualizer
By using **Visual M2M Data Visualizer**, you can confirm that the data is being communicated in real time. If you import the "SCREEN file (.scrn)" and "DAT file (.dat)" in the same directory as this notebook into **Visual M2M Data Visualizer**, you can check the data as follows. (For details, see the Operation Manual of **Visual M2M Data Visualizer**.)
 
 
On the screen below, `Acceleration raw` panel shows the data before conversion (data sent by Motion app), and `Acceleration Converted` panel shows the moving average calculated using `intdash-py`.

<img src="https://github.com/aptpod/intdash-py-sample-codes/blob/master/img/img3.png?raw=true">